## _*Using Qiskit Aqua for exact cover problems*_

In mathematics, given a collection $S$ of subsets of a set $X$.
An exact cover is a subcollection $S_{ec} \subseteq S$ such that each element in $X$ is contained in exactly one subset $\in S_{ec}$. 

We will go through three examples to show:
1. How to run the optimization using the declarative approach
2. How to run the optimization using the programmatic approach
3. How how to run the optimization with the VQE.

#### The problem and the brute-force method.

First, let us take a look at the list of subsets.

In [1]:
import numpy as np
import json

from qiskit import BasicAer
from qiskit.aqua import run_algorithm
from qiskit.aqua.input import EnergyInput
from qiskit.aqua.translators.ising import exact_cover
from qiskit.aqua.algorithms import ExactEigensolver

input_file = 'sample.exactcover'
with open(input_file) as f:
    list_of_subsets = json.load(f)
    print(list_of_subsets)

[[2, 3, 4], [1, 2], [3, 4], [1, 2, 3]]


Then we apply the brute-force method. Basically, we exhaustively try all the binary assignments. In each binary assignment, the entry of a subset is either 0 (meaning the subset is not in the cover) or 1 (meaning the subset is in the cover). We print the binary assignment that satisfies the definition of the exact cover.

In [2]:
def brute_force():
    # brute-force way: try every possible assignment!
    has_sol = False

    def bitfield(n, L):
        result = np.binary_repr(n, L)
        return [int(digit) for digit in result]  # [2:] to chop off the "0b" part

    L = len(list_of_subsets)
    max = 2**L
    for i in range(max):
        cur = bitfield(i, L)
        cur_v = exact_cover.check_solution_satisfiability(cur, list_of_subsets)
        if cur_v:
            has_sol = True
            break
    return has_sol, cur

has_sol, cur = brute_force()
if has_sol:
    print("Solution is", cur)
else:
    print("No solution is found")

Solution is [0, 1, 1, 0]


#### Part I: Run the optimization using the declarative approach

Here the steps are:
* Create the qubit operator i.e. Ising Hamiltonian, using `exact_cover` ising translator
* Create an EnergyInput object and a dictionary describing the algorithm and the components for Aqua to solve the problem
* Run the algorithm and get the result
* Use the result with the `exact_cover` object to determine a solution

In [3]:
qubit_op, offset = exact_cover.get_exact_cover_qubitops(list_of_subsets)
algo_input = EnergyInput(qubit_op)

params = {
    'problem': {'name': 'ising'},
    'algorithm': {'name': 'ExactEigensolver'}
}
result = run_algorithm(params, algo_input)

x = exact_cover.sample_most_likely(len(list_of_subsets), result['eigvecs'][0])
ising_sol = exact_cover.get_solution(x)
np.testing.assert_array_equal(ising_sol, [0, 1, 1, 0])
if exact_cover.check_solution_satisfiability(ising_sol, list_of_subsets):
    print("Solution is", ising_sol)
else:
    print("No solution is found")

Solution is [0. 1. 1. 0.]


#### Part II: Run the optimization using the programmatic approach

The main difference here is running the Aqua algorithm. Here we directly construct the algorithm and then run() it to get the result. The post computation on the result is identical.

In [4]:
algo = ExactEigensolver(algo_input.qubit_op, k=1, aux_operators=[])
result = algo.run()

x = exact_cover.sample_most_likely(len(list_of_subsets), result['eigvecs'][0])
ising_sol = exact_cover.get_solution(x)
np.testing.assert_array_equal(ising_sol, [0, 1, 1, 0])

if exact_cover.check_solution_satisfiability(ising_sol, list_of_subsets):
    print("Solution is", ising_sol)
else:
    print("No solution is found")

Solution is [0. 1. 1. 0.]


#### Part III: Run the optimization with the VQE

##### Declarative

We reuse the EnergyInput object we created above. VQE algorithm needs an optimizer and a variational form. Then also we need a quantum backend on which the algorithm will run.

In [5]:
params = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': {'name': 'VQE'},
    'optimizer': {'name': 'COBYLA'},
    'variational_form': {'name': 'RYRZ', 'depth': 5}
}
backend = BasicAer.get_backend('statevector_simulator')
result = run_algorithm(params, algo_input, backend=backend)

x = exact_cover.sample_most_likely(len(list_of_subsets), result['eigvecs'][0])
ising_sol = exact_cover.get_solution(x)
if exact_cover.check_solution_satisfiability(ising_sol, list_of_subsets):
    print("Solution is", ising_sol)
else:
    print("No solution is found")

Solution is [0. 1. 1. 0.]


##### Programmatic

We can create the objects directly ourselves too and run VQE for the result

In [6]:
from qiskit.aqua import aqua_globals
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua.components.variational_forms import RY

aqua_globals.random_seed = 10598

optimizer = COBYLA()
var_form = RY(qubit_op.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubit_op, var_form, optimizer)

backend = BasicAer.get_backend('statevector_simulator')
result = vqe.run(backend)

x = exact_cover.sample_most_likely(len(list_of_subsets), result['eigvecs'][0])
ising_sol = exact_cover.get_solution(x)
if exact_cover.check_solution_satisfiability(ising_sol, list_of_subsets):
    print("Solution is", ising_sol)
else:
    print("No solution is found")

Solution is [0. 1. 1. 0.]
